In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm

## Cargue de datos

In [85]:
data = pd.read_csv("data/data_01.txt", sep="\t")
df_prom = data.copy()

# EDA

In [86]:
print(data.shape)
data.head(10)

(775466, 3)


,fecha,idTerminal,oper
0,2017-01-31 08:19:33.000,1828,0
1,2017-01-31 08:47:16.000,1828,1
2,2017-01-31 08:48:07.000,1828,1
3,2017-01-31 08:48:58.000,1828,1
4,2017-01-31 08:49:53.000,1828,1
5,2017-01-31 08:51:35.000,1593,0
6,2017-01-31 08:52:45.000,1593,1
7,2017-01-31 08:53:33.000,1593,1
8,2017-01-31 08:53:53.000,1746,0
9,2017-01-31 08:54:33.000,1593,1


Se observa que se cuenta con un total de 775.466 registros de transacciones.

In [87]:
data.dtypes

fecha         object
idTerminal     int64
oper           int64
dtype: object

Al revisar el tipo de datos, se observa que la columna fecha no está en el formato adecuado, por lo que se requiere hacer el respectivo ajuste. Además, sería util transformar las variables de ___idTerminal___ y ___oper___ a string por motivos de visualización.

In [88]:
data['fecha'] = pd.to_datetime(data['fecha'])
data['idTerminal'] = data['idTerminal'].astype(str)
data['oper'] = data['oper'].astype(str)
data.set_index('fecha', inplace=True)

### Verificación datos nulos

In [92]:
data.isna().sum()

idTerminal    0
oper          0
dtype: int64

### Histórico de transacciones quincenales

In [89]:
df_grouped = data.resample('1W').size().reset_index(name='count')
fig = px.line(df_grouped, x='fecha', y='count', title='Histórico de transacciones',
              labels={'fecha': 'Fecha', 'count': 'Transacciones'})
fig.show()

De acuerdo con el gráfico, se puede observar que el histórico de transacciones no tiene una tendencia clara en el periodo de 4 meses analizados. Se observa cierta volatilidad en el volumen transaccional, y aunque en los primeros meses registran algunos picos y caidas recurrentes, estas no se mantienen despues de principios de abril. Por último, se observa que hacia la ultima semana se mayo se presenta una fuerte caida, ubicando el volumen transaccional en su punto más bajo del periodo analizado.

### Promedio de transacciones diarias por terminal y tipo de operación

In [125]:
df_prom['fecha'] = pd.to_datetime(df_prom['fecha'])
df_prom['fecha'] = df_prom['fecha'].dt.date
df_prom['idTerminal'] = df_prom['idTerminal'].astype(str)
df_prom['oper'] = df_prom['oper'].astype(str)

df_agrupado = df_prom.groupby(['idTerminal', 'oper', 'fecha']).size().reset_index(name='Transacciones')

pivot_table = df_agrupado.pivot_table(
    values='Transacciones', 
    index=['idTerminal', 'oper'], 
    aggfunc='mean'
).reset_index()


terms = ["1774", "1908", "1964", "1910", "1980"]
ops = ["0", "3", "1", "7", "4"]

pivot_filtrado = pivot_table[
    pivot_table['idTerminal'].isin(terms) & 
    pivot_table['oper'].isin(ops)]

pivot_filtrado

,idTerminal,oper,Transacciones
378,1774,0,147.024793
379,1774,1,28.308333
399,1774,3,54.933884
403,1774,4,10.715517
408,1774,7,10.789916
660,1908,0,131.140496
661,1908,1,20.391667
675,1908,3,52.462810
680,1908,4,19.909091
684,1908,7,16.533333


In [126]:
fig = px.line(pivot_filtrado, x='idTerminal', y='Transacciones', color='oper', markers='o', title="Promedio de transacciones diarias por terminal y tipo de transacción")
fig.show()

c:\Users\jmoncada43\OneDrive - Cementos Argos S.A\Documentos\Platzi\DataScience\Prueba\env\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Los resultados indican como la operación 0 tiene una demanda diaria promedio muy superior respecto a las demás operaciones, rondando un promedio por terminal entre 100 y 150 operaciones diarias. Por otro lado, las otras 4 operaciones con mayor demanda muestran niveles promedio similares, situandose en un rango diario entre las 8 y 50 operaciones aproximadamente.

### Histograma

In [90]:
df_grouped = data.resample('2W').size().reset_index(name='count')
fig = px.histogram(df_grouped, x='count', nbins=6, title='Histograma de transacciones quincenales',
                   labels={'count': 'Transacciones'})

fig.update_layout(width=600, height=400) 
fig.update_yaxes(title_text="Recuento quincenal")
fig.show()

Al análizar el histograma, se observa que durante 7 periodos quincenales se registro un volumen de entre 80.000 y 100.000 transacciones, siendo este el intervalo más recurrente. Se destaca que en ninguna quincena se presentó un volumen entre las 60.000 y 80.000 transacciones, pues el restante se ubica entre las 20.000 y 60.000 y las 100.000 y 120.000

### Curva de distribución normal

In [91]:
df_grouped = data.resample('2W').size().reset_index(name='count')
media = df_grouped['count'].mean()
desviacion_estandar = df_grouped['count'].std()

x_values = np.linspace(df_grouped['count'].min(), df_grouped['count'].max(), 1000)
y_values = norm.pdf(x_values, media, desviacion_estandar)


fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Distribución normal'))
fig.update_layout(title='Curva de la distribución normal',
                  xaxis_title='Transacciones',
                  yaxis_title='Densidad de probabilidad',
                  showlegend=True)
fig.update_layout(width=600, height=400) 

fig.show()

De acuerdo con la curva de la distribución normal, se puede observar que el volumen transaccional tiene un comportamiento aproximadamente normal con un sesgo a la derecha, indicando que los datos se concentran hacia la cola derecha de la distribución.

Se detalla que el conjunto de datos analizados no contiene datos nulos o vacios.

### Transacciones en el tiempo

In [93]:
period_table = data.resample('2W').size().reset_index(name='# operaciones')
period_table.rename(columns={'index': 'quincena'}, inplace=True)
period_table

,fecha,# operaciones
0,2017-02-05,46585
1,2017-02-19,83570
2,2017-03-05,101521
3,2017-03-19,88492
4,2017-04-02,88562
5,2017-04-16,84447
6,2017-04-30,89011
7,2017-05-14,87809
8,2017-05-28,80403
9,2017-06-11,25066


In [94]:
fig = px.bar(period_table, x=period_table['fecha'].astype(str), y='# operaciones')
fig.update_layout(xaxis={'tickmode': 'array', 'tickvals': period_table['fecha'].astype(str).unique(), 'ticktext': period_table['fecha'].astype(str).unique()},
                    width=600, height=400)
fig.show()

Al obervar el volumen transaccional de forma quincenal, se observa que en la primera quincena del año el volumen es considerablemente bajo respecto a los meses siguientes, sin embargo, la ultima quincena es la que presenta un valor más bajo en volumen, alcanzando cerca de 25.000 transacciones. En las demás quincenas, se observa cierta estabilidad en el volumen con pequeñas fluctuaciones.

### Transacciones por terminal

In [95]:
term_table = data.pivot_table(index='idTerminal', aggfunc='size', fill_value=0).reset_index(name='# operaciones').set_index('idTerminal').sort_values(
    by='# operaciones', ascending=False)
term_table['Pareto (%)'] = round(term_table['# operaciones'].cumsum() / term_table['# operaciones'].sum() * 100, 2)
term_table

,# operaciones,Pareto (%)
idTerminal,,
1774,32122,4.14
1908,30933,8.13
1964,29999,12.00
1910,26789,15.45
1980,25243,18.71
1741,23278,21.71
1593,23014,24.68
1824,22295,27.55
1864,21649,30.35


In [96]:
fig=px.bar(term_table, x=term_table.index, y='# operaciones', title='Transacciones por terminal')
fig.show()

Al observar las transacciones por terminal, se destaca que las terminales _1774_, _1908_, _1964_, _1910_ y _1980_ son las que registran un mayor volumen transaccional con 	_32.122_, _30.933_, _29.999_, _26.789_ y _25.243_, respectivamente. Al realizar un análisis de pareto, se observa que estas 5 terminales abarcan cerca el 18.7% del total de transacciones registradas, destacandose así su relevancia entre las terminales elegidas para realizar transacciones.

Por otro lado, las terminales con una menor demanda son las terminales _1515_ y _1507_, registrando en total _4.325_ y _4.496_ transacciones, respectivamente. Lo cual indica el bajo uso de estas terminales para la realización de transacciones.

### Transacciones por tipo de operación

In [97]:
import pandas as pd

ops_table = data.pivot_table(index='oper', aggfunc='size', fill_value=0).reset_index(name='# operaciones').set_index('oper').sort_values(
    by='# operaciones', ascending=False)
ops_table['Pareto (%)'] = round(ops_table['# operaciones'].cumsum() / ops_table['# operaciones'].sum() * 100, 2)
ops_table


,# operaciones,Pareto (%)
oper,,
0,423929,54.67
3,127399,71.10
1,79195,81.31
7,50662,87.84
4,35817,92.46
5,16124,94.54
2,7526,95.51
10,6988,96.41
6,6066,97.19


In [98]:
fig=px.bar(ops_table, x=ops_table.index, y='# operaciones', title="Transacciones por tipo de operación")
fig.show()

Al observar las transacciones por tipo de operación, se destaca que las operaciones _0_, _3_, _1_, _7_ y _4_ son las que registran un mayor volumen transaccional con _423.929_, _127.399_, _79.195_, _50.662_ y _35.817_, respectivamente. Al realizar un análisis de pareto, se observa que estas 5 operaciones abarcan cerca el 92.5% del total de transacciones registradas, destacandose así su relevancia entre las terminales elegidas para realizar transacciones. De hecho, la operación 0 tiene la particularidad de abarcar el 54.67% de las operaciones.

Por otro lado, las operaciones con una menor demanda son las _33_ y _45_, registrando cada una tan solo una transaccion. Lo cual indica el bajo uso de estas operaciones.

### Volumen por terminal y tipo de operación

In [99]:
pivot_table = data.pivot_table(index='idTerminal', columns='oper', aggfunc='size', fill_value=0)
pivot_table = pivot_table.loc[pivot_table.sum(axis=1).sort_values(ascending=False).index]
pivot_table = pivot_table[pivot_table.sum().sort_values(ascending=False).index]
pivot_table

oper,0,3,1,7,4,5,2,10,6,9,...,41,27,40,42,37,30,43,44,33,45
idTerminal,,,,,,,,,,,,,,,,,,,,,
1774,17790,6647,3397,1284,1243,169,29,139,354,210,...,1,0,0,0,0,0,0,1,0,0
1908,15868,6348,2447,1984,2409,394,22,469,357,189,...,1,0,0,0,0,0,0,0,1,0
1964,15620,6413,3317,1173,1864,279,354,157,293,159,...,0,0,0,0,0,0,1,0,0,0
1910,16395,3781,1888,2279,870,502,84,224,143,142,...,2,0,0,0,0,0,0,1,0,0
1980,12266,3380,4159,1657,695,430,1815,127,133,102,...,0,0,0,0,0,3,0,0,0,0
1741,11266,5576,2341,1129,1537,375,303,134,269,141,...,0,0,0,0,0,0,0,0,0,0
1593,12420,3475,2773,1526,785,777,126,306,186,96,...,0,1,0,0,0,0,0,0,0,0
1824,11388,4875,2087,860,1664,364,45,139,320,137,...,0,1,0,0,0,0,0,0,0,0
1864,10379,4340,1839,2154,1183,501,13,341,253,168,...,1,0,0,0,0,0,0,0,0,0


In [100]:
pivot_table = pivot_table.reset_index()
df_long = pd.melt(pivot_table, id_vars=['idTerminal'], var_name='oper', value_name='count')
fig = px.bar(df_long, x='idTerminal', y='count', color='oper', barmode='stack', title="Transacciones por terminal y tipo de operación")
fig.show()

c:\Users\jmoncada43\OneDrive - Cementos Argos S.A\Documentos\Platzi\DataScience\Prueba\env\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Al obervar el volumen transaccional se observa que a nivel general las terminales cuentan con la capacidad de atender todos los tipos de operaciones. 
Además, se destaca como en la mayoria de estas terminales predominan las operaciones _0_, _3_, _1_, _7_ y _4_, las cuales como vimos anteriormente representan el 92.5% del total. Además, se observa como el resto de operaciones tienen un volumen considerablemente bajo en todas las terminales.

In [101]:
opers = ["0", "1", "3", "4", "7"]
df_long['oper'] = df_long['oper'].apply(lambda x: x if x in opers else 'otros')
fig = px.bar(df_long, x='idTerminal', y='count', color='oper', barmode='stack')
fig.show()

c:\Users\jmoncada43\OneDrive - Cementos Argos S.A\Documentos\Platzi\DataScience\Prueba\env\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Al generar un agrupamiento de las operaciones con bajo volumen transaccional, se destaca aun mejor la importancia que tienen las operaciones _0_, _3_, _1_, _7_, especialemnte la primera, la cual predomina en todas las terminales.

# Conclusión

En resumen, se observa una serie de operaciones que tinen un volumen transaccional muy superior a las demás, estas son: 
_0_, _3_, _1_, _7_.

Además, se destaca como estas predominan en todas las terminales, incluidas las terminales con mayor demanda:
_1774_, _1908_, _1964_, _1910_ y _1980_.

En consecuencia, aportaria un gran valor a la organización establecer un pronostico que aproxime la demanda por estas operaciones dentro de las terminales en mención, con la idea de establecer las estrategias correspondientes para garantizar un adecuado funcionamiento de las terminales y sus modulos de servicio.

# Observaciones sobre la calidad y cantidad de los datos

### Calidad:
De acuerdo con el análisis, se observa que se cuenta con una base de datos sencilla, con 3 campos que no cuentan con campos vacíos o nulos. Se observ además que los registros son consistentes, es decir, manejan una estructura definida que evita varios pasos de pre-procesamiento y pasar al modelamiento mucho más rapido. Como posible mejora se podria agregar un mejor contexto respecto al escenario en que los datos son obtenidos.

### Cantidad:
Si bien se cuenta con un amplio volumen de transacciones, considero que se hace necesario tener información con un lapso de tiempo mucho más amplio, esto con la idea de identificar si el comportamiento transaccional está sujeto a alguna estacionalidad de mediano o largo plazo, lo cual permitiria tener un pronostico más exacto y que tenga en cuenta periodos de crecimiento o decrecimiento transaccional (p.e: diciembre y enero, respectivamente). Además, dado que el propósito es establecer el pronostico para cada terminal, cuando se termina haciendo el agrupamiento de datos se termina contando con pocos registros para cada terminal, lo cual influye significativamente en el desempeño de los modelos.
Una mayor ventana temporal permitira ejecutar modelos de series de tiempo que pueden llegar a tener un muy buen desempeño como los ARIMA o SARIMA.

### Modelos seleccionados
Para la realización del pronostico se tuvieron en cuenta inicialmente 4 modelos:

- SARIMA
- Random Forest
- Support Vector Machine
- Gradient Boosting

No obstante, el modelo SARIMA fue descartado por las restricciones comentadas anteriormente. Se decidio mantener los modelos de Random Forest y Gradient Boosting debido a que su estructura de ensamble es una herramienta poderosa para tener resultados más precisos. El modelo SVM se considera especialmente por su función de kernel y la flexibilidad para modelar relaciones lineales y no lineales.

Los resultados de los modelos, así como el pronóstico son expuestos en el cuaderno ___model.ipynb___ 